# Test Clustering from Edges

We have limited performance when counting the number of correct edges against the number of faulty edges. However, this does not indicate that we have poor performance when evaluating out clusters. Even though we have some proportion of faulty edges, due to our high number of correct edges we might have a strong ability to cluster sequences. Let's try.

In this case we'll need adjacency matrices. Let's test the performance in a single region first.

## Imports

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import time
from itertools import repeat
import multiprocessing
from joblib import Parallel, delayed
import pandas as pd
from sklearn.metrics import auc
import matplotlib
import random

import math
from scipy.linalg import *
from sklearn.cluster import KMeans

from sklearn.cluster import SpectralCoclustering


1. sketch with k = 20 roughly
2. euclidean 

In [2]:
"""
Convert from numpy array to pandas df for plotting
"""
def numpy_counts_to_pandas(array):
    df = pd.DataFrame()
    df["true_pos"] = array[0,:]
    df["tot_pos"] = array[1,:]
    df["false_pos"] = array[2,:]
    df["tot_neg"] = array[3,:]

    df["tpr"] = df["true_pos"] / df["tot_pos"]
    df["fpr"] = df["false_pos"] / df["tot_neg"]
    df["band_size"] = range(1,500)

    return df

"""
Plot the ROC curve
"""
def plotROC(df, title = "ROC Curve"):
    bands = df["band_size"]
    tprs = df["tpr"]
    fprs = df["fpr"]

    #area_under_curve = auc(fprs, tprs)

    # Plot the ROC curve
    fig, ax = plt.subplots()

    plt.scatter(fprs, tprs, c = bands, marker = ".")
    num_labels = min(10, len(bands))
    scale = int(len(bands)//num_labels)
    for i in range(num_labels):
        index = i*scale
        ax.text(fprs[index], tprs[index]+0.05, bands[index])

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc = "lower right")

    plt.show()

"""
Plot the ROC curve for multiple different paramters
"""
def plotROCwithParams(dfs, params, title = "ROC Curve with params", param_header = ""):
    if len(dfs) is not len(params):
        print("Number of inputs and parameters is not the same!")
        return

    fig, ax = plt.subplots()

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)

    # Add y=x line
    plt.plot([0, 1], [0, 1], 'k--')

    cmap = matplotlib.cm.get_cmap('plasma')
    norm = plt.Normalize(vmin=0, vmax=len(params))

    for i, df in enumerate(dfs):
        bands = df["band_size"]
        tprs = df["tpr"]
        fprs = df["fpr"]

        # Plot the ROC curve
        plt.scatter(fprs, tprs, 
                    c = cmap(norm(i)), 
                    marker = ".",
                    label= param_header + f"{params[i]}"
        )

        # Add AUC label TODO
        #area_under_curve = auc(fprs, tprs)

        # Add band size labels
        #num_labels = min(10, len(bands))
        #scale = int(len(bands)//num_labels)
        #for i in range(num_labels):
        #    index = i*scale
        #    ax.text(fprs[index], tprs[index]+0.05, bands[index])

    plt.legend(loc = "upper left", bbox_to_anchor = (1,0,.5,1))
    plt.show()

"""
Returns sorted seqs and labels by labels from fastq file
"""
def readFastq(fastqFile:str):
    seqs :list[str] = []
    labels :list[str] = []
    with open(fastqFile,"r") as f:
        i = 0
        curHap = -1
        for line in f.readlines():
            if i % 4 == 0:
                for segment in line.strip().split(" "):
                    if segment.startswith("HP"):
                        curHap = segment.split(":")[-1]
            elif i % 4 == 1:
                if curHap == -1:
                    continue
                curSeq = line.strip()
                seqs.append(curSeq)
                labels.append(curHap)
                curHap = -1
                curSeq = ""
            i += 1
    seqs, labels = zip(*sorted(zip(seqs, labels), key = lambda x : x[1]))
    return seqs, labels

"""
Iterate over all band sizes for a single region. Called in parallel over every region at once.
"""
def test_bands(signature_matrix_directory, file, band_sizes, include_length = False, include_hash = False):

    signature_matrix = np.load(os.path.join(signature_matrix_directory, file))
    labels = signature_matrix[0,:].astype('int8')
    signature_matrix = signature_matrix[1:,:]

    if include_length or type(include_hash) is int:
        fastq_file = os.path.join("../../../output/HG002_20bp/all_chr/", file[:-3]) + "fastq"
        seqs, _ = readFastq(fastq_file)
        lengths = [len(x) for x in seqs]
        if 0 in lengths:
            return 0
    else:
        seqs = None
        lengths = None

    # Return Adjacency Matrix in this case
    if type(band_sizes) is int:
        adjacency_matrix = tprSignatureMatrix(signature_matrix, band_sizes, labels, include_length, 
            include_hash, seqs, lengths, want_adjacency_matrix = True)
        return adjacency_matrix, labels
    # Return TPR FPR information
    else:
        counts = np.zeros((4, len(band_sizes)), dtype = int)
        for band_size in band_sizes:
            counts[:,band_size-1] = tprSignatureMatrix(
                signature_matrix, 
                band_size, 
                labels, 
                include_length = include_length, 
                include_hash = include_hash, 
                seqs = seqs,
                lengths = lengths
            )
        return counts

"""
Generate TPR and FPR for a given signature matrix, band length, and label list
Returns [test_pos, tot_pos, test_neg, tot_neg]
"""
def tprSignatureMatrix(signature_matrix, band_length, labels, 
                       include_length = False, include_hash = False, 
                       seqs = None, lengths = 0, want_adjacency_matrix = False):

    def comparator_length(include_length, a_len, b_len):
        # Absolute Threshold
        if include_length[1] == 0:
            return abs(a_len-b_len) <= include_length[0]
        # Percentage Threshold
        else:
            return abs(a_len-b_len) / max(a_len,b_len) <= include_length[0]

    def comparator_hash(include_hash, a, b, a_len, b_len):
        min_len = min(a_len, b_len)
        num_compare = max(int(min_len * (include_hash / 100)), 1)
        indices = np.random.choice(min_len, num_compare, replace=False)
        a_subseq = np.array(list(a))[indices]
        b_subseq = np.array(list(b))[indices]
        return hash(a_subseq.tobytes()) == hash(b_subseq.tobytes())

    def connectBucket(bucket, adjacencyMatrix):
        for i in range(len(bucket)-1):
            for j in range(i+1, len(bucket)):
                if include_length and not comparator_length(include_length, lengths[i], lengths[j]):
                    continue
                if type(include_hash) is int and not comparator_hash(include_hash, seqs[i], seqs[j], lengths[i], lengths[j]):
                    continue
                adjacencyMatrix[bucket[i],bucket[j]] = True
                adjacencyMatrix[bucket[j],bucket[i]] = True
        return adjacencyMatrix

    firstOcc = np.argmax(labels>1)
    adjacencyMatrix = np.zeros((len(labels), len(labels)))

    for i in range(signature_matrix.shape[0]//band_length):
        buckets = defaultdict(list[int])
        startBucket = i*band_length
        endBucket = (i+1)*band_length
        for j in range(signature_matrix.shape[1]):
            buckets[signature_matrix[startBucket:endBucket, j].tobytes()].append(j)
        for bucket in buckets.values():
            adjacencyMatrix = connectBucket(bucket, adjacencyMatrix)
    if want_adjacency_matrix:
        return adjacencyMatrix

    return tpr_fpr(len(labels), firstOcc, adjacencyMatrix)

"""
Generate true positive rate and false positive rate for a given adjacency matrix
"""
def tpr_fpr(n, firstOcc, adjacencyMatrix):
    trueMask = np.zeros((n , n), dtype = bool)
    trueMask[:firstOcc, :firstOcc] = 1
    trueMask[firstOcc:, firstOcc:] = 1

    true_pos = int(np.sum(adjacencyMatrix, where = trueMask)/2)
    tot_pos = int((np.sum(trueMask) - n)/2)
    false_pos = int(np.sum(adjacencyMatrix, where = ~trueMask)/2)
    tot_neg = int(np.sum(~trueMask)/2)

    local_counts = np.array([true_pos, tot_pos, false_pos, tot_neg])

    return local_counts

"""
Callable function to parallelize banding step
    if len(band_size) == 1 then returns adjacency matrices
"""
def parallel_regions(signature_matrix_directory, band_sizes = range(1, 500), verbose = 0,
                     non_repetitive = False, repetitive = False, 
                     include_hash = False, include_length = False,
                     fastq_file = False):

    if fastq_file:
        signature_matrix_file = fastq_file.replace("fastq", "npy")
        adjacency_matrix = test_bands(signature_matrix_directory, signature_matrix_file, band_sizes)
        return adjacency_matrix

    # Get correct numpy file set
    if repetitive or non_repetitive:
        def bed_to_npy(bed):
            chrom, start, end = bed.split("\t")
            return f"{chrom}_{int(start)-20}-{int(end)+20}.npy"
        repetitive_regions_file = "../../../output/analysis/human_chm13v2.0_maskedY_rCRS.trf.bed"
        with open(repetitive_regions_file, "r") as f:
            repetitive_regions = set([bed_to_npy(x) for x in f.read().splitlines()])
    if non_repetitive:
        numpy_files = [x for x in os.listdir(signature_matrix_directory) if x not in repetitive_regions]
    if repetitive:
        numpy_files = [x for x in os.listdir(signature_matrix_directory) if x in repetitive_regions]
    if repetitive and non_repetitive or not repetitive and not non_repetitive:
        numpy_files = os.listdir(signature_matrix_directory)

    inputs = zip(repeat(signature_matrix_directory), 
                 numpy_files, 
                 repeat(band_sizes), 
                 repeat(include_length), 
                 repeat(include_hash)
    )
    num_cores = multiprocessing.cpu_count()

    print("Running Parallel Step")
    start = time.time()
    outputs = Parallel(n_jobs=num_cores, verbose=verbose)(delayed(test_bands)(i, j, k, l, m) for i, j, k, l, m  in inputs)
    print("Total time\t:", time.time() - start)

    counts = outputs[0]
    for output in outputs[1:]:
        counts += output
    return counts

"""
Return index of first occurrence of label 2
"""
def get_first_occ(labels):
    labelSet = set(labels)
    firstOcc = [0 for _ in range(len(labelSet))]
    for i, item in enumerate(sorted(labelSet)):
        firstOcc[i] = labels.index(item)
    return firstOcc[1]


## Function Calls

In [20]:
fastq_file = "chr15_30271921-30272041.fastq"
cosine_signature_matrix_directory = "../../../output/signatureMtxs_20bp/sketch/1000,21,1"
cosine_band_size = 200

euclidean_signature_matrix_directory = "../../../output/signatureMtxs_20bp/euclidean/1000,21,8,1"
euclidean_band_size = 50

adjacency_matrix, labels = parallel_regions(cosine_signature_matrix_directory, cosine_band_size, fastq_file = fastq_file)
first_occurrence = get_first_occ(list(labels))

In [21]:
counts = tpr_fpr(len(adjacency_matrix), first_occurrence, adjacency_matrix)
print("tpr:", counts[0]/counts[1])
print("fpr:", counts[2]/counts[3])

tpr: 0.06074766355140187
fpr: 0.03619909502262444


## Visualize a Region